In [2]:
from __future__ import annotations
from SPARQLWrapper import SPARQLWrapper, JSON
import os
import sys
import datetime
import dateutil
import json
import pathlib
import time
from collections import defaultdict
from typing import Any, Callable, Dict, Iterable, List, Tuple, Optional, Set, Union

import elasticsearch
import pandas as pd
import requests
import requests_cache
import spacy
import yfinance as yf
from dotenv import load_dotenv
from elasticsearch_dsl import connections, Document, Date, Keyword, Q, Search, Text, Range, Integer
from flashtext import KeywordProcessor

In [1]:
%cd /workspace/entity_knowledge_in_bert/bert_entity
from vocab import Vocab
from data_loader_wiki import EDLDataset

class args:
    data_version_name = "dummy"
    train_loc_file = "train.loc"
    device = "cpu"
    label_size = 1024
    uncased = True
    train_data_dir = "data"
    wiki_lang_version = "enwiki"
    batch_size = 1
    data_workers = 4
    collect_most_popular_labels_steps = 1
    top_rnns = False
    project = False
    entity_embedding_size = 768
    sparse = True
    out_device = 'cpu'
    finetuning = 3

vocab = Vocab(args)
train_dataset = EDLDataset(
    args, split="train", vocab=vocab, device=args.device, label_size=args.label_size)
iterator = train_dataset.get_data_iter(args, batch_size=args.batch_size, vocab=vocab, train=True)

for x in iterator:
#     batch_token_ids, label_ids, label_probs, eval_mask, _, _, orig_batch, loaded_batch = x
#     print(x)
#     print()
    break

/workspace/entity_knowledge_in_bert/bert_entity


In [7]:
batch_token_ids, label_ids, label_probs, eval_mask, _, _, orig_batch, loaded_batch = x

# batch_token_ids, label_ids
# orig_batch
# eval_mask
# loaded_batch
# label_probs.shape
# label_ids.shape
# batch_token_ids.shape
# label_ids
# for p in label_probs[0]:
#     print(p)
# label_probs[0, 13]
# for p in label_probs[0]:
#     print(p.sum())


In [3]:
from model import Net
model = Net(args, vocab.size())

In [4]:
# print(model)
y = model(batch_token_ids, label_ids, label_probs)

tensor([ 1312, 51457,   184,  ..., 32567, 10045, 44862])


In [80]:
logits, _, y_hat, _, _, enc = y
# logits.shape
# logits
# y_hat


# vocab.size()
label_ids.

torch.Size([1024])

In [9]:
from pytorch_pretrained_bert import BertModel

bert = BertModel.from_pretrained("bert-base-uncased")

In [59]:
bert.eval()
encoded_layers, _ = bert(batch_token_ids)
# batch_token_ids.shape
enc = encoded_layers[-1]

In [57]:
import torch.nn as nn
out = nn.Embedding(num_embeddings=vocab.size(), embedding_dim=768, sparse=True)
out.eval()

Embedding(51459, 768, sparse=True)

In [74]:
# _.shape
# encoded_layers[-1].shape
embeded_labels = out(label_ids) # (1024, 768)
embeded_labels = embeded_labels.transpose(0, 1)  # (768, 1024)
logits = enc.matmul(embeded_labels) # (1, 254, 1024)
# logits.argmax(-1)  # (1,254)
logits[0,0,13]

tensor(41.4637, grad_fn=<SelectBackward>)

In [5]:
import torch
import torch.nn.functional as F
x = torch.randn(1, 5)  # (1, 5)
y = torch.randint(5, (1,), dtype=torch.int64)  # (1, )
loss = F.cross_entropy(x, y)
# loss
# x.shape, y.shape
# x, y

x = torch.randn((1, 5))
y = torch.rand((1, 5))
# F.binary_cross_entropy(torch.sigmoid(x), y)
# x, y
# F.binary_cross_entropy(x, y)
# torch.sigmoid(x)
y
y.to("cpu")

tensor([[0.3413, 0.5543, 0.3464, 0.6351, 0.4112]])

In [3]:
def count_corpus_mentions_cooccurs(mentions: Dict[str, List[str]]) -> Any:
    mention2idx = get_mention2idx_dict(mentions)

    def count_cooccur(bag: Iterable[int], cooccur: Dict[int, Dict[int, int]] = defaultdict(lambda: defaultdict(int))) -> Dict[int, Dict[int, int]]:
        for a in bag:
            for b in bag:
                cooccur[a][b] += 1
        return cooccur

    nlp = spacy.load('en_core_web_sm')
    processor = KeywordProcessor()
    processor.add_keywords_from_dict(mentions)

    sent_cooccur, atk_cooccur = defaultdict(
        lambda: defaultdict(int)), defaultdict(lambda: defaultdict(int))
    atk_bags = []
    for i, hit in enumerate(es.scan_scraper_page("*cnbc*")):
        if i % 100 == 0:
            print(i)
        if i > 1000:
            break
        sents = []
        try:
            sents.append(hit.article_title)
        except:
            pass
        try:
            sents += [sent.text for sent in nlp(hit.article_text).sents]
        except:
            pass

        atk_bag = set()
        for sent in sents:
            kws = processor.extract_keywords(sent, span_info=False)
            sent_bag = [mention2idx[kw] for kw in kws]
            sent_cooccur = count_cooccur(sent_bag, sent_cooccur)
            atk_bag |= set(sent_bag)
        atk_cooccur = count_cooccur(atk_bag, atk_cooccur)
        atk_bags.append(atk_bag)

    return (
        # 在一個句子中，兩個entity同時出現的次數: {a: {a: 4, b: 2, c:1}, b: {...}}
        dict(sent_cooccur),
        # 在一篇文章中，兩個entity同時出現的次數: {a: {a: 4, b: 2, c:1}, b: {...}}
        dict(atk_cooccur),
        # 在一篇文章中出現的entities集合成一個bag: [{a, b, c}, {a, c, f}, ...]
        set(map(frozenset, atk_bags)),)

In [5]:
filepath = "/workspace/entity_knowledge_in_bert/app/outputs/wiki_mentions.json"

with open(filepath, encoding='utf-8') as f:
    mentions = json.load(f)

mentions

FileNotFoundError: [Errno 2] No such file or directory: '/workspace/entity_knowledge_in_bert/app/outputs/wiki_mentions.json'

In [13]:
def get_lines():
    with gzip.open("/workspace/entity_knowledge_in_bert/app/downloads/cirrus/aawiki-20200907-cirrussearch-general.json.gz", 'r') as f:        
        for i, ln1 in enumerate(f):
            print(i)
            ln2 = next(f)
            yield i, ln1, ln2

g = get_lines()
next(g)
next(g)

0
1


(1,
 b'{"index":{"_type":"page","_id":"1847"}}\n',
 b'{"template":[],"redirect":[],"content_model":"wikitext","heading":[],"source_text":"<\\/div>\\n<\\/div>\\n<\\/div><!-- END STYLE INFORMATION --><noinclude>\\n<\\/noinclude>","version_type":"external","opening_text":null,"wiki":"aawiki","coordinates":[],"auxiliary_text":[],"language":"aa","title":"ReyBrujo\\/Styles\\/Footer","version":2992,"external_link":[],"namespace_text":"User","namespace":2,"text_bytes":74,"incoming_links":0,"text":"","category":[],"defaultsort":false,"outgoing_link":[],"timestamp":"2007-03-19T00:57:49Z","create_timestamp":"2007-03-19T00:57:49Z"}\n')

In [2]:
import time
import gzip
import threading, queue

class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return self.it.__next__()

def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

@threadsafe_generator
def read_line():
    with gzip.open("/workspace/entity_knowledge_in_bert/app/downloads/cirrus/aawiki-20200907-cirrussearch-general.json.gz", 'r') as f:        
        for i, ln1 in enumerate(f):
            print(i)
            ln2 = next(f)
            yield i, ln1, ln2

q = queue.Queue()
readline = read_line()
            
def worker(thread_i):
    while True:
        i, ln1, ln2 = q.get()
        print(f'Working on {thread_i} : {i}')
#         print(f'Finished {item}')
        time.sleep(3)

        i, ln1, ln2 = next(readline)
        q.put((i, ln1, ln2))
        q.task_done()

for i in range(5):
    threading.Thread(target=worker,args=(i,)).start()
#     threading.Thread(target=wrapper_targetFunc,
#                      args=(targetFunction, q, otherarg)).start()

for _ in range(10):
    i, ln1, ln2 = next(readline)
    q.put((i, ln1, ln2))

print('All task requests sent')

# block until all tasks are done
q.join()
print('All work completed')

0
1
2
3
Working on 2 : 0
Working on 0 : 1
4
5
6
7
8
9
All task requests sent
Working on 3 : 2
Working on 4 : 3
Working on 1 : 4
10
Working on 1 : 5
11
Working on 2 : 6
12
Working on 0 : 7
13
Working on 1 : 8
14
Working on 4 : 9
15
Working on 3 : 10
16
Working on 1 : 11
17
Working on 2 : 12
18
Working on 0 : 13
19
Working on 4 : 14
20
Working on 3 : 15
21
Working on 1 : 16
22
Working on 1 : 17
23
Working on 2 : 18
24
Working on 0 : 19
25
Working on 4 : 20
26
Working on 3 : 21
27
Working on 1 : 22
28
Working on 1 : 23
29
Working on 2 : 24
30
Working on 0 : 25
31
Working on 3 : 26
32
Working on 4 : 27
33
Working on 1 : 28


KeyboardInterrupt: 

In [9]:
# EDGAR
# !pip install secedgar
# !pip install python-xbrl
!pip install ixbrlparse

  Created wheel for ixbrlparse: filename=ixbrlparse-0.1.2-py3-none-any.whl size=8174 sha256=76f9c2416adc0a6304771f209eccc12016aa41d11420a72c06b34b07a83cc9c1
  Stored in directory: /root/.cache/pip/wheels/54/f1/7d/469ed056196dee602624dceb93767639d3739d3e0e92911a86
Successfully built ixbrlparse


In [1]:
from secedgar.filings import Filing, FilingType

filings = Filing(cik_lookup='aapl', filing_type=FilingType.FILING_10Q, count=1) # 10Q filings for Apple (ticker "aapl")
# my_filings.save('~/path/to/dir')
filings.save("/workspace/entity_knowledge_in_bert/app/downloads/edgar")

ConnectionError: HTTPConnectionPool(host='www.sec.gov', port=80): Max retries exceeded with url: /cgi-bin/browse-edgar?action=getcompany&CIK=aapl (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ff9c9f536d0>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [4]:
import xml.etree.ElementTree as ET
f = "/workspace/entity_knowledge_in_bert/app/downloads/0001477333-20-000032.txt"
tree = ET.parse(f)
tree


ParseError: mismatched tag: line 46, column 2 (<string>)

In [4]:
# from xbrl import XBRLParser, GAAP, GAAPSerializer, DEISerializer
# xbrl_parser = XBRLParser()
# xbrl = xbrl_parser.parse(open("/workspace/entity_knowledge_in_bert/app/downloads/edgar/aapl/10-q/0000320193-20-000062.txt"))
dei_obj = xbrl_parser.parseDEI(xbrl)

# Serialize the DEI data
serializer = DEISerializer()
result = serializer.dump(dei_obj)

# Print out the serialized DEI data
print(result)

{'public_float': 0.0, 'shares_outstanding': 4275634000.0, 'trading_symbol': 'AAPL', 'company_name': 'Apple Inc.'}


In [3]:
gaap_obj = xbrl_parser.parseGAAP(xbrl,doc_date="",context="current",ignore_errors=1)
serializer = GAAPSerializer()
result = serializer.dump(gaap_obj)
print(result)

{'net_cash_flows_investing_discontinued': 0.0, 'nonoperating_income_loss': 0.0, 'comprehensive_income_interest': 0.0, 'common_shares_outstanding': 0.0, 'net_cash_flows_operating_continuing': 0.0, 'liabilities_and_equity': 0.0, 'net_cash_flows_financing': 0.0, 'net_cash_flows_investing': 0.0, 'net_cash_flows_discontinued': 0.0, 'common_shares_issued': 0.0, 'liabilities': 0.0, 'interest_and_debt_expense': 0.0, 'net_income_shareholders': 0.0, 'preferred_stock_dividends': 0.0, 'net_income_parent': 0.0, 'stockholders_equity': 0.0, 'operating_expenses': 0.0, 'assets': 0.0, 'equity': 0.0, 'redeemable_noncontrolling_interest': 0.0, 'other_operating_income': 0.0, 'net_cash_flows_financing_continuing': 0.0, 'other_comprehensive_income': 0.0, 'temporary_equity': 0.0, 'net_income_loss': 0.0, 'noncurrent_liabilities': 0.0, 'operating_income_loss': 0.0, 'income_before_equity_investments': 0.0, 'current_liabilities': 0.0, 'equity_attributable_interest': 0.0, 'common_shares_authorized': 0.0, 'income_t

In [11]:
from ixbrlparse import IXBRL
with open("/workspace/entity_knowledge_in_bert/app/downloads/edgar/aapl/10-q/0000320193-20-000062.txt") as f:
    xbrl = IXBRL(f, raise_on_error=False)

In [28]:
# xbrl.contexts['i656f36c938ba410686db6183a7b1d75c_D20190929-20200627']
a = xbrl.numeric[0]
# a.value
a = xbrl.nonnumeric[0]
a.value

'Cupertino'

In [56]:
from bs4 import BeautifulSoup
p = "/workspace/entity_knowledge_in_bert/app/downloads/0001477333-20-000032.txt"
with open(p) as f:
    soup = BeautifulSoup(f, 'lxml-xml')

In [57]:
# soup.find("DOCUMENT").TYPE
soup.find_all("DOCUMENT")

[<DOCUMENT>
 <TYPE>8-K
 <SEQUENCE>1
 <FILENAME>net202008068k.htm
 <DESCRIPTION>8-K
 <TEXT>
 </TEXT></DESCRIPTION></FILENAME></SEQUENCE></TYPE></DOCUMENT>]

In [58]:
!pip install pyedgar

     |████████████████████████████████| 40 kB 16 kB/s eta 0:00:01
  Created wheel for pyedgar: filename=pyedgar-0.1.5-py3-none-any.whl size=44711 sha256=f7b02a9a9f00781cbb303f27f65a812aa2dfe0276a855263c048b00b478ca884
  Stored in directory: /root/.cache/pip/wheels/c7/90/6a/6eaceb342a838c3ac6c0860c808568e56044763793712fc061
Successfully built pyedgar


In [27]:
from pyedgar import Filing
# f = Filing(cik=1477333, accession='0001628280-20-012347')
f = Filing(cik=1131181, accession="0001131181-20-000005")
print(f)

<EDGAR filing (1131181/0001131181-20-000005) Headers:False, Text:False, Documents:False>


In [35]:
from bs4 import BeautifulSoup
print(len(f.documents))
soup = BeautifulSoup(f.documents[1]['full_text'], 'lxml-xml')

2


In [49]:
list(soup.find('infoTable').children)

['\n',
 <ns1:nameOfIssuer>ALIBABA GROUP HLDG LTD</ns1:nameOfIssuer>,
 '\n',
 <ns1:titleOfClass>SPONSORED ADS</ns1:titleOfClass>,
 '\n',
 <ns1:cusip>01609W102</ns1:cusip>,
 '\n',
 <ns1:value>5107</ns1:value>,
 '\n',
 <ns1:shrsOrPrnAmt>
 <ns1:sshPrnamt>23681</ns1:sshPrnamt>
 <ns1:sshPrnamtType>SH</ns1:sshPrnamtType>
 </ns1:shrsOrPrnAmt>,
 '\n',
 <ns1:investmentDiscretion>SOLE</ns1:investmentDiscretion>,
 '\n',
 <ns1:votingAuthority>
 <ns1:Sole>0</ns1:Sole>
 <ns1:Shared>0</ns1:Shared>
 <ns1:None>23681</ns1:None>
 </ns1:votingAuthority>,
 '\n']

In [15]:
# print(f.type, f)
# print(len(f.documents))
# f.urls
# f.documents[70]
import json

for d in f.documents:
    if d['type'] == 'JSON':
        j = json.loads(d['full_text'])
        break 

In [26]:
# j['R1']
j['instance']['cloud-20200630.htm']['report']['R2']

{'firstAnchor': {'ancestors': ['span',
   'td',
   'tr',
   'table',
   'div',
   'body',
   'html'],
  'baseRef': 'cloud-20200630.htm',
  'contextRef': 'i695c9105f12241a9aa9f4d469bd4a9b3_I20200630',
  'decimals': '-3',
  'first': True,
  'lang': 'en-US',
  'name': 'us-gaap:CashAndCashEquivalentsAtCarryingValue',
  'reportCount': 1,
  'unitRef': 'usd',
  'xsiNil': 'false'},
 'groupType': 'statement',
 'isDefault': 'false',
 'longName': '1001002 - Statement - CONDENSED CONSOLIDATED BALANCE SHEETS',
 'role': 'http://www.cloudflare.com/role/CONDENSEDCONSOLIDATEDBALANCESHEETS',
 'shortName': 'CONDENSED CONSOLIDATED BALANCE SHEETS',
 'subGroupType': '',
 'uniqueAnchor': {'ancestors': ['span',
   'td',
   'tr',
   'table',
   'div',
   'body',
   'html'],
  'baseRef': 'cloud-20200630.htm',
  'contextRef': 'i695c9105f12241a9aa9f4d469bd4a9b3_I20200630',
  'decimals': '-3',
  'lang': 'en-US',
  'name': 'us-gaap:AvailableForSaleSecuritiesDebtSecuritiesCurrent',
  'reportCount': 1,
  'unique': Tr

In [5]:
import json
p = "/workspace/entity_knowledge_in_bert/app/downloads/symbols.json"
with open(p) as f:
    j = json.load(f)

In [10]:
for e in j:
    if e['type'] == 'rt':
        print(e)

{'symbol': 'AGBAR', 'exchange': 'NAS', 'name': 'AGBA Acquisition Ltd Rights For Shares', 'date': '2020-10-17', 'type': 'rt', 'iexId': 'IEX_4A485A464D572D52', 'region': 'US', 'currency': 'USD', 'isEnabled': True, 'figi': 'BBG00PPX10R3', 'cik': '1769624'}
{'symbol': 'ALACR', 'exchange': 'NAS', 'name': 'Alberton Acquisition Corp Rights For Shares', 'date': '2020-10-17', 'type': 'rt', 'iexId': 'IEX_47384B3042522D52', 'region': 'US', 'currency': 'USD', 'isEnabled': True, 'figi': 'BBG00MMCKZC7', 'cik': '1748621'}
{'symbol': 'ANDAR', 'exchange': 'NAS', 'name': 'Andina Acquisition Corp. III Rights For Shares', 'date': '2020-10-17', 'type': 'rt', 'iexId': 'IEX_56583931534E2D52', 'region': 'US', 'currency': 'USD', 'isEnabled': True, 'figi': 'BBG00NJ41KD0', 'cik': '1691936'}
{'symbol': 'BMY^', 'exchange': 'NYS', 'name': 'Bristol-Myers Squibb Company Contingent Value Rights 2019-31.12.2020', 'date': '2020-10-17', 'type': 'rt', 'iexId': 'IEX_484A4E484B432D52', 'region': 'US', 'currency': 'USD', 'is

In [3]:
import json
from qwikidata.entity import WikidataItem
from qwikidata.json_dump import WikidataJsonDump
from qwikidata.utils import dump_entities_to_json

dump_path = "/workspace/entity_knowledge_in_bert/app/downloads/latest-all.json.bz2"

for i, entity_dict in enumerate(WikidataJsonDump(dump_path)):
    if entity_dict["type"] == "item":
        e = WikidataItem(entity_dict)
        print(e.entity_id, e.get_enwiki_title())
#         print(json.dumps(e._entity_dict))
        dumped = json.dumps(e._entity_dict)
        break

j = json.loads(dumped)
e = WikidataItem(entity_dict)
print(e.entity_id, e.get_enwiki_title())

Q31 Belgium
Q31 Belgium


In [5]:
a = [1,2,3]
f = filter(lambda e: e > 2, a)
print([e for e in f])
print([e for e in f])

[3]
[]
